# ![University Logo](https://redintercol.net/images/Logo_EAFIT.svg.png)
## **Trabajo 2**
### Mariel Viviana Sánchez
### Laura Riveros
### Maria del Rosario Castro Mantilla

#**Paso 2:**
Realizar preparación y representación de documentos basado en LLMs, revisar y
escoger algún modelo pre-entrenado de embeddings para texto (ej: openai, hugging face, cohere, etc). Una vez creada la representación vectorial de los textos en embeddings, realizar la indexación y almacenamiento en una base de datos vectorial, de preferencia una de las más sencillas como chromadb (https://www.trychroma.com/ ) u otra más robusta como opensearch (https://opensearch.org/) o Pinecone (https://www.pinecone.io/)

In [ ]:
!pip install transformers torch chromadb

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch


# Ejemplo con un modelo de Hugging Face
# Updated model_name to include the organization
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [ ]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv('/content/drive/MyDrive/EAFIT/ALMACENAMIENTO Y RECUPERACIÓN INFO/TRABAJOS/TRABAJO 2/FINAL/climateTwitterData.csv')  # Especifica la ruta a tu archivo

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/EAFIT/ALMACENAMIENTO Y RECUPERACIÓN INFO/TRABAJOS/TRABAJO 2/FINAL/climateTwitterData.csv'

In [ ]:
print(df.head())

   Unnamed: 0.1  Unnamed: 0            id     author_id  \
0             0           0  1.211810e+18  7.590000e+17   
1             1           1  1.210670e+18  2.219547e+07   
2             2           2  1.210590e+18  1.070000e+18   
3             3           3  1.210260e+18  1.339821e+09   
4             4           4  1.209640e+18  1.339821e+09   

                                                text  retweets  \
0  2020 is the year we #votethemout, the year we ...        15   
1  Winter has not stopped this group of dedicated...         9   
2  WEEK 55 of #ClimateStrike at the @UN. Next wee...       545   
3   A year of resistance, as youth protests shape...         1   
4   HAPPY HOLIDAYS #greta #gretathunberg #climate...         1   

                                           permalink  \
0  https://twitter.com/Sphiamia/status/1211807074...   
1  https://twitter.com/StephDujarric/status/12106...   
2  https://twitter.com/AlexandriaV2005/status/121...   
3  https://twitter.com/E

In [ ]:
documentos = df['text'].tolist()

In [ ]:
# Función para obtener embeddings
def get_embedding(text):
    # Tokenize the input text
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # Get the model's output
    with torch.no_grad():  # Disable gradient calculation during inference
        outputs = model(**inputs)

    # Use the 'pooler_output' or 'last_hidden_state' for embeddings
    # Here, we use 'pooler_output' which is a sentence-level embedding
    embeddings = outputs.pooler_output

    return embeddings.cpu().numpy()  # Convert to numpy array

# Generar embeddings para todos los documentos
embeddings = [get_embedding(doc) for doc in documentos]

In [ ]:
# Convertir la lista de embeddings a un DataFrame
import numpy as np
embeddings_file_path = "/content/drive/MyDrive/EAFIT/ALMACENAMIENTO Y RECUPERACIÓN INFO/TRABAJOS/TRABAJO 2/embeddings.npy"

try:
    embeddings_data = np.load(embeddings_file_path, allow_pickle=True)
    # Load existing embeddings from file if it exists
    # Reshape embeddings_data to 2D if necessary
    embeddings_data = embeddings_data.reshape(embeddings_data.shape[0], -1)
except FileNotFoundError:
    embeddings_data = embeddings # Use previously calculated embeddings if file not found
    # Reshape if embeddings_data is 3D
    if embeddings_data[0].ndim > 1:
        # The issue is here: embeddings_data is a list of 2D arrays,
        # we need to convert it to a 2D array.
        embeddings_data = np.vstack(embeddings_data)
        # Or, flatten each sub-array if you want a single row per document:
        # embeddings_data = [emb.flatten() for emb in embeddings_data]


embeddings_df = pd.DataFrame(embeddings_data)  # Create DataFrame with the correct embeddings data

# Agregar los documentos como una columna
embeddings_df['document'] = documentos

# Reorganizar las columnas (opcional)
embeddings_df = embeddings_df[['document'] + [col for col in embeddings_df.columns if col != 'document']]

# Guardar el DataFrame como un archivo CSV
embeddings_df.to_csv('/content/drive/MyDrive/EAFIT/ALMACENAMIENTO Y RECUPERACIÓN INFO/TRABAJOS/TRABAJO 2/embeddings.csv', index=False)

In [ ]:
# Inicializar ChromaDB
!pip install chromadb # install chromadb using pip
from chromadb import Client # import the necessary classes
client = Client() # Initialize ChromaDB without deprecated parameters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 49.5 MB/s eta 0:00:0

In [ ]:
collection_name = "my_collection"
collection = client.create_collection(name=collection_name)

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
embeddings_file_path = '/content/drive/MyDrive/EAFIT/ALMACENAMIENTO Y RECUPERACIÓN INFO/TRABAJOS/TRABAJO 2/embeddings.csv'
data = pd.read_csv(embeddings_file_path)

In [ ]:
data.head()

,document,0,1,2,3,4,5,6,7,8,...,374,375,376,377,378,379,380,381,382,383
0,"2020 is the year we #votethemout, the year we ...",0.001072,0.071700,0.058674,0.048360,-0.141348,-0.032065,0.071684,0.110727,-0.065337,...,-0.049462,0.064445,-0.038205,0.049707,-0.061472,-0.044861,0.063604,0.030028,-0.070776,-0.006586
1,Winter has not stopped this group of dedicated...,0.012677,0.020616,0.062577,0.027512,-0.141822,-0.025485,0.008640,0.130931,-0.069043,...,0.002719,0.101389,-0.069118,0.060340,-0.067699,-0.028403,0.032605,0.072798,-0.033148,0.015330
2,WEEK 55 of #ClimateStrike at the @UN. Next wee...,-0.002418,0.058622,0.035351,0.026319,-0.142267,-0.104064,-0.004875,0.086556,-0.063340,...,-0.025863,0.059744,-0.059716,0.047644,-0.069324,-0.033608,0.079455,0.103062,-0.061225,-0.008846
3,"A year of resistance, as youth protests shape...",0.033822,0.054196,0.042111,0.103010,-0.179311,-0.067262,0.054253,0.140015,-0.052258,...,-0.052647,0.102112,-0.045644,0.036647,-0.048927,-0.116487,0.046410,0.048982,-0.057769,0.011411
4,HAPPY HOLIDAYS #greta #gretathunberg #climate...,-0.013019,-0.038367,0.060412,0.062417,-0.106921,-0.075596,-0.006015,0.093874,-0.074673,...,-0.037271,0.039069,-0.098994,0.032062,-0.048734,-0.068678,0.036154,0.059160,-0.065346,-0.037547


In [ ]:
embeddings_df = data.iloc[:, :-1]
print(embeddings_df)

                                                document         0         1  \
0      2020 is the year we #votethemout, the year we ...  0.001072  0.071700   
1      Winter has not stopped this group of dedicated...  0.012677  0.020616   
2      WEEK 55 of #ClimateStrike at the @UN. Next wee... -0.002418  0.058622   
3       A year of resistance, as youth protests shape...  0.033822  0.054196   
4       HAPPY HOLIDAYS #greta #gretathunberg #climate... -0.013019 -0.038367   
...                                                  ...       ...       ...   
72400  No more straws at LBM... only if YOU ask for i...  0.058096 -0.021366   
72401  My #Trumps may not believe in #climatechange b... -0.011894 -0.006411   
72402  Time is over. Act now!!! #Friday4Future #Clima... -0.008748  0.028781   
72403  This is my first contribution on visualizing t...  0.032043  0.034432   
72404  "This assault on science is an outrage and sho...  0.000711  0.011643   

              2         3         4    

In [ ]:
# Replace NaN values with 0 in your DataFrame before creating embeddings list
embeddings_df = data.drop(columns=['document'], errors='ignore').fillna(0)  # Replace NaN with 0, ignore if 'document' not found

# Now create the embeddings list
embeddings_list = embeddings_df.values.tolist()
documents_list = data['document'].tolist() if 'document' in data.columns else []  # Handle case where 'document' is missing
ids = [str(i) for i in range(len(documents_list))]

# Now create the embeddings list
embeddings_list = embeddings_df.values.tolist()
documents_list = data['document'].tolist()
ids = [str(i) for i in range(len(documents_list))]

# Get the maximum batch size for your ChromaDB client
max_batch_size = 41665  # Or use client.get_max_batch_size() if available

# Add data in batches using list slicing
for i in range(0, len(ids), max_batch_size):
    batch_ids = ids[i : i + max_batch_size]
    batch_embeddings = embeddings_list[i : i + max_batch_size]
    batch_documents = documents_list[i : i + max_batch_size]

    collection.add(
        ids=batch_ids,
        embeddings=batch_embeddings,
        documents=batch_documents,
    )
    print(f"Added batch of size {len(batch_ids)}")  # Print progress

In [ ]:
results = collection.query(
    query_texts=["clim"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
print(results)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 99.8MiB/s]


{'ids': [['57850', '61105']], 'distances': [[2.209016799926758, 2.209016799926758]], 'metadatas': [[None, None]], 'embeddings': None, 'documents': [['“The [bushfire crisis] is not a new normal. It’s a step to a new normal which will be worse than what we’re seeing now” @petrmatous on the role of #climatechange in exacerbating prevalence and intensity of #bushfires ', '“The [bushfire crisis] is not a new normal. It’s a step to a new normal which will be worse than what we’re seeing now” @petrmatous on the role of #climatechange in exacerbating prevalence and intensity of #bushfires ']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


In [ ]:
results = collection.query(
    query_texts=["Winter has not stopped"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
print(results)

{'ids': [['57850', '61105']], 'distances': [[2.063502073287964, 2.063502073287964]], 'metadatas': [[None, None]], 'embeddings': None, 'documents': [['“The [bushfire crisis] is not a new normal. It’s a step to a new normal which will be worse than what we’re seeing now” @petrmatous on the role of #climatechange in exacerbating prevalence and intensity of #bushfires ', '“The [bushfire crisis] is not a new normal. It’s a step to a new normal which will be worse than what we’re seeing now” @petrmatous on the role of #climatechange in exacerbating prevalence and intensity of #bushfires ']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


In [ ]:
results = collection.query(
    query_texts=["WEEK 55 of #ClimateStrike"], # Chroma will embed this for you
    n_results=1 # how many results to return
)
print(results)

{'ids': [['61105']], 'distances': [[2.0863683223724365]], 'metadatas': [[None]], 'embeddings': None, 'documents': [['“The [bushfire crisis] is not a new normal. It’s a step to a new normal which will be worse than what we’re seeing now” @petrmatous on the role of #climatechange in exacerbating prevalence and intensity of #bushfires ']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


In [11]:
results = collection.query(
    query_texts=["#Colombia"], # Chroma will embed this for you
    n_results=1 # how many results to return
)
print(results)

{'ids': [[]], 'distances': [[]], 'metadatas': [[]], 'embeddings': None, 'documents': [[]], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}
